In [1]:
import torch

if torch.cuda.is_available():
    print("CUDA is available. GPU in use:", torch.cuda.get_device_name(0))
else:
    print("CUDA is NOT available. Using CPU.")

CUDA is NOT available. Using CPU.


In [3]:
# importing google flan-t5-small for translation refinement
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer, AutoModelForCausalLM

# c2t model loading example
import whisper_s2t
from whisper_s2t.backends.ctranslate2.model import BEST_ASR_CONFIG

#preprocessing audio
import importlib
import split_and_preprocess
importlib.reload(split_and_preprocess)
from split_and_preprocess import process_chunks
import merge_audio
importlib.reload(merge_audio)
from merge_audio import merge_wav_files
import os
import librosa

In [4]:
# determine the device
device = "cuda" if torch.cuda.is_available() else "cpu"

### Loading the whisper model

In [5]:
ct2_model = whisper_s2t.load_model(
    model_identifier="large-v2", 
    backend='CTranslate2', 
    compute_type='int8',  # Best for cpu
    device=device,
    asr_options=BEST_ASR_CONFIG,
)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

e:\anaconda3\envs\mals2t\lib\site-packages\whisper_s2t\speech_segmenter\frame_vad.py:74: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast()
e:\anaconda3\envs\mals2t\lib\site-packages\torch\amp\autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


### Loading google flan-t5-large

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")

### Deepseek R1 1.5B

In [6]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
model_r1 = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

e:\anaconda3\envs\mals2t\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in E:\.cache\huggingface\hub\models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:01<?, ?B/s]

#### Preprocessing

In [7]:
audio_path = "megha.aac"  # Path to your input audio file
temp_dir = "temp_chunks"            # Directory to save processed chunks

audio, sr = librosa.load(audio_path, sr=None)

chunk_paths, sr = process_chunks(audio_path, temp_dir=temp_dir, chunk_duration=30)
print(f"Processed chunk files: {chunk_paths}")

C:\Users\user\AppData\Local\Temp\ipykernel_6244\1795977638.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(audio_path, sr=None)
e:\anaconda3\envs\mals2t\lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
c:\Users\user\Desktop\Programming\Internships\The marble jar\mal_speech_translation\split_and_preprocess.py:36: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(input_path, sr=None, mono=True)
e:\anaconda3\envs\mals2t\lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Processed and saved 3 chunks to temp_chunks
Processed chunk files: ['temp_chunks\\chunk_0_processed.wav', 'temp_chunks\\chunk_1_processed.wav', 'temp_chunks\\chunk_2_processed.wav']


In [8]:
merged_audio_path = "merged_output.wav"  # Path to save the merged audio file
merge_wav_files(temp_dir, merged_audio_path)

Merged 3 files into merged_output.wav


### Translation using whisper model

In [10]:
# temp_dir = "temp_chunks"
lang_codes = ['ml']
initial_prompts = [None]
# Direct Malayalam-to-English translation
tasks = ['translate']
files = ["merged_output.wav"]  # Use the merged audio file

# chunk_paths = sorted(
#     [os.path.join(temp_dir, f) for f in os.listdir(temp_dir) if f.endswith('.wav')]
# )

results = ct2_model.transcribe_with_vad(
    files,
    lang_codes=lang_codes,
    tasks=tasks,
    initial_prompts=initial_prompts,
    batch_size=32  # or higher if your hardware allows
)
# all_text = [result[0]['text'] for result in results]
# full_transcript = " ".join(all_text)
print(results[0][0]['text'])

Transcribing: 100%|██████████| 100/100 [03:40<00:00,  2.21s/it]

Before IK Kerala, Kerala was divided into four parts. Thiruvidhankur, Kochi, Malabar, South Kannara. The people of these villages were of the same culture, language and way of life. Kerala was formed by combining them. In 1920, the Nagpur Congress decided to form the committee of the organization in a bilingual state. In 1921,


### Further refining translated text using another llm

In [ ]:
# Craft a prompt with instructions and context
input_text = (
    "You are an expert translator who can translate malayalam to english. "
    "Given the following sentence, make it more sensible and fluent in English. "
    "If the sentence does not make sense, use the words in the sentence to form the most plausible translation that makes sense. "
    "The translation must be such that if its translated back to malayalam, we get a similar sentence as the original malayalam sentence. "
    "Here is the sentence:\n"
    f"{results[0][0]['text']}\n"
    "Improved translation:"
)
# Generate the improved translation
input_ids = tokenizer(input_text, return_tensors="pt").to(device)
outputs = model_r1.generate(**input_ids)
improved_translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(tokenizer.decode(outputs[0]))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [ ]:
prompt = f"Paraphrase this to correct and fluent English: {full_transcript}"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))